In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 72.7 MB/s 
     |████████████████████████████████| 101 kB 13.9 MB/s 
     |████████████████████████████████| 6.6 MB 66.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 


In [4]:
import os
import gc
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold,StratifiedGroupKFold,GroupKFold
from sklearn.metrics import log_loss,f1_score

from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
INPUT_DIR = '/content/drive/MyDrive/Competitions/Signate/Student Cup 2022/input/'
OUTPUT_DIR = '/content/drive/MyDrive/Competitions/Signate/Student Cup 2022/output/'
OUTPUT_SUB_DIR = os.path.join(OUTPUT_DIR,'Submission')
OUTPUT_MODEL_DIR = os.path.join(OUTPUT_DIR,'Model/DeBERTa-base[ver5]/')

In [6]:
class CFG:
    num_workers=2
    path=OUTPUT_MODEL_DIR
    config_path=OUTPUT_MODEL_DIR+'config.pth'
    model="microsoft/deberta-v3-base"
    batch_size=16
    dropout=0.1
    target_size=4
    max_len=1024
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    gradient_checkpointing=True
    freezing=True

In [7]:
# Loss Func
def criterion(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

def softmax(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div
"""
def get_score(y_true, y_pred):
    y_pred = softmax(y_pred)
    score = log_loss(y_true, y_pred)
    return round(score, 5)
"""
def get_score(outputs, labels):
    outputs = F.softmax(torch.tensor(outputs)).numpy()
    return f1_score(np.argmax(outputs,axis=1),labels ,average='macro')

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [8]:
oof_df = pd.read_pickle(OUTPUT_MODEL_DIR+'oof_df.pkl')
labels = oof_df['label'].values
preds = oof_df[['Consultant','Data scientist','Machine learning engineer','Software engineer']].values.tolist()
score = get_score(preds, labels)
LOGGER.info(f'CV Score: {score:<.4f}')

CV Score: 0.8107
INFO:__main__:CV Score: 0.8107


In [9]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False
        
def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)
            
    return freezed_parameters

def set_embedding_parameters_bits(embeddings_path, optim_bits=32):
    """
    https://github.com/huggingface/transformers/issues/14819#issuecomment-1003427930
    """
    
    embedding_types = ("word", "position", "token_type")
    for embedding_type in embedding_types:
        attr_name = f"{embedding_type}_embeddings"
        
        if hasattr(embeddings_path, attr_name): 
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                getattr(embeddings_path, attr_name), 'weight', {'optim_bits': optim_bits}
            )

In [10]:
test = pd.read_csv(os.path.join(INPUT_DIR, 'test.csv'))
submission_df = pd.read_csv(os.path.join(INPUT_DIR, 'submit_sample.csv'),header=None)

def remove_tag(x):
    p = re.compile(r"<[^>]*?>")
    return p.sub('',x)

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_tag(text)
        #アルファベット以外をスペースに置き換え
        #clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        clean_texts.append(text)
    return clean_texts



from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text


test['description'] = cleaning(test['description'])
test['inputs'] = test['description'].apply(lambda x : resolve_encodings_and_normalize(x))
display(test.head())

,id,description,inputs
0,1516,Building decision-making models and proposing ...,Building decision-making models and proposing ...
1,1517,Educate homeowners on the benefits of solar en...,Educate homeowners on the benefits of solar en...
2,1518,"Design, develop, document, and implement web a...","Design, develop, document, and implement web a..."
3,1519,Apply advanced technical expertise and skills ...,Apply advanced technical expertise and skills ...
4,1520,Project manage and deliver against our roadmap...,Project manage and deliver against our roadmap...


In [11]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
class Dataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = CFG.max_len
        self.text = df['inputs'].values
        self.tokenizer = CFG.tokenizer
        #self.targets = df['label'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length = self.max_len
        )
        samples = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            #'target': self.targets[index]
        }

        if 'token_type_ids' in inputs:
            samples['token_type_ids'] = inputs['token_type_ids']
            
        return samples

In [13]:
class Collate:
    def __init__(self, tokenizer, isTrain=True):
        self.tokenizer = tokenizer
        self.isTrain = isTrain
        # self.args = args

    def __call__(self, batch):
        output = dict()
        output["input_ids"] = [sample["input_ids"] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        if self.isTrain:
            output["target"] = [sample["target"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["input_ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["input_ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["input_ids"]]
            output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in output["attention_mask"]]
        else:
            output["input_ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["input_ids"]]
            output["attention_mask"] = [(batch_max - len(s)) * [0] + s for s in output["attention_mask"]]

        # convert to tensors
        output["input_ids"] = torch.tensor(output["input_ids"], dtype=torch.long)
        output["attention_mask"] = torch.tensor(output["attention_mask"], dtype=torch.long)
        if self.isTrain:
            output["target"] = torch.tensor(output["target"], dtype=torch.long)

        return output
    
collate_fn = Collate(CFG.tokenizer, isTrain=False)

In [14]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9) #
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [15]:
class CustomModel(nn.Module):
    def __init__(self, model_name):
        super(CustomModel, self).__init__()
        # Header (fast or normal)
        self.model = AutoModel.from_pretrained(model_name)
        
        # Gradient_checkpointing
        if CFG.gradient_checkpointing:
            (self.model).gradient_checkpointing_enable()
        
        # Freezing
        if CFG.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            CFG.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())
        
        self.config = AutoConfig.from_pretrained(model_name)
        self.drop = nn.Dropout(p=CFG.dropout)
        self.pooler = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, CFG.target_size)
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids, 
                         attention_mask=mask,
                         output_hidden_states=False)
        out = self.pooler(out.last_hidden_state, mask)
        out = self.drop(out)
        outputs = self.fc(out)
        return outputs

In [16]:
def inference_one_epoch(model, dataloader, device):
    model.eval()
    pred = []
    model.to(device)
    for step, data in enumerate(dataloader):
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        with torch.no_grad():
            outputs = model(ids, mask)
        pred.append(outputs.to('cpu').numpy())
    pred = np.concatenate(pred)
    return pred

In [17]:
testdataset = Dataset(test, CFG.tokenizer, CFG.max_len)

test_loader = DataLoader(testdataset, 
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         collate_fn = collate_fn,
                         num_workers = CFG.num_workers,
                         pin_memory = True,
                         drop_last = False,
                         )

predictions = []

for fold in CFG.trn_fold:
    model = CustomModel(CFG.model)
    config_path=CFG.config_path
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location = torch.device('cpu'))
    model.load_state_dict(state['model'])

    prediction = inference_one_epoch(model, test_loader, device)
    prediction = F.softmax(torch.tensor(prediction)).numpy().astype(float)
    predictions.append(prediction)
    del model, state, prediction
    gc.collect()
    torch.cuda.empty_cache()

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the m

In [18]:
sub1 = submission_df.copy()
sub1.columns = ["id","label"]

#method1
sub1_predictions = np.mean(predictions, axis=0)
sub1['Data scientist'] = sub1_predictions[:, 0]
sub1['Machine learning engineer'] = sub1_predictions[:, 1]
sub1['Software engineer'] = sub1_predictions[:, 2]
sub1['Consultant'] = sub1_predictions[:, 3]
sub1["label"] = np.argmax(sub1_predictions,axis=1)


sub1["label"] = sub1["label"].astype("int")
sub1["label"] = sub1["label"] + 1
sub1[["id","label"]].to_csv(os.path.join(OUTPUT_SUB_DIR,"8107submission26.csv"),index=False,header=False)
display(sub1)

,id,label,Data scientist,Machine learning engineer,Software engineer,Consultant
0,1516,1,0.967974,0.003798,0.003026,0.025203
1,1517,4,0.004859,0.000203,0.000936,0.994002
2,1518,3,0.002020,0.003112,0.992704,0.002163
3,1519,4,0.007792,0.000118,0.000543,0.991547
4,1520,3,0.031511,0.013799,0.945695,0.008995
...,...,...,...,...,...,...
1512,3028,3,0.033661,0.010779,0.927701,0.027859
1513,3029,1,0.980067,0.004257,0.001694,0.013983
1514,3030,3,0.001965,0.006254,0.989892,0.001889
1515,3031,4,0.272849,0.001106,0.003544,0.722501


In [19]:
sub1.loc[sub1.id==2301]

,id,label,Data scientist,Machine learning engineer,Software engineer,Consultant
785,2301,1,0.974115,0.012445,0.005588,0.007853


In [20]:
#train,testでダブっているデータを参照
dup_test_ids = [1707,2122,2291,2775,2191,1700,2304,2149,2676,2844,2144,2764,1774,2446,2736,2301,1822,1852,2070,1609,2423,2695,
                2077,2409,2233,2076,1568,3001,1662,1997, 2896,2352,2842,2321,1630,2259,2968,1551, 1673, 2168]
sub1[sub1["id"].isin(dup_test_ids)]                

,id,label,Data scientist,Machine learning engineer,Software engineer,Consultant
35,1551,4,0.126799,0.001053,0.003023,0.869125
52,1568,1,0.981770,0.003014,0.001679,0.013537
93,1609,4,0.196569,0.000771,0.001087,0.801573
114,1630,4,0.038873,0.001436,0.006925,0.952766
146,1662,1,0.958301,0.006668,0.007253,0.027778
157,1673,4,0.126799,0.001053,0.003023,0.869125
184,1700,1,0.968507,0.003078,0.001984,0.026431
191,1707,3,0.020195,0.011207,0.932070,0.036528
258,1774,4,0.004261,0.000098,0.000461,0.995180
306,1822,4,0.003297,0.000092,0.000445,0.996166


In [21]:
T = pd.DataFrame([[1707,2122,2291,2775,2191,1700,2304,2149,2676,2844,2144,2764,1774,2446,2736,2301,1822,1852,2070,1609,2423,2695,
                2077,2409,2233,2076,1568,3001,1662,1997, 2896,2352,2842,2321,1630,2259,2968,1551, 1673, 2168],
                [3,4,1,4,1,1,1,1,3,1,3,4,4,1,1,1,4,4,3,4,4,4,3,4,4,1,1,1,1,3,3,1,4,4,4,1,4,4,4,4]]).T
T.columns =["id","label"]
T = T.sort_values(by="id")
T

,id,label
37,1551,4
26,1568,1
19,1609,4
34,1630,4
28,1662,1
38,1673,4
5,1700,1
0,1707,3
12,1774,4
16,1822,4
